In [0]:

print("Hello")

In [0]:

unified_df = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-today.v1/fact/granularity=daily/year=2021/")
unified_df.printSchema()

In [0]:

from pyspark.storagelevel import StorageLevel
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType

unified = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-featured/unified/featured.ios14-today.v1/fact/granularity=daily/")
unified.persist(StorageLevel.MEMORY_AND_DISK)
unified.createOrReplaceTempView("unified")

sfOptions = {
  "sfURL" : "appannie_aa_int_prod.us-east-1.snowflakecomputing.com",
  "sfUser" : "app_bdp_data_validation_qa",
  "sfPassword" : "0HN#s@Wa5$1R8jVj",
  "sfDatabase" : "AA_INTELLIGENCE_PRODUCTION",
  "sfSchema" : "ADL_STORE_FREE",
  "sfWarehouse" : "wh_dod_read7"
}
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
db = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query",  "select * from FACT_FEATURED_IOS14_APPS_V1_CLUSTER_BY_DATE") \
  .load()
db.persist(StorageLevel.MEMORY_AND_DISK)
db.createOrReplaceTempView("db")


In [0]:


_date = '2021-05-01'
breakdowns = ['country_code', 'device_code', 'featured_story_key', 'product_key']
for breakdown in breakdowns:
    sd = spark.sql(f"select {breakdown}, count(distinct product_key) as cnt1 from unified where date='{_date}' group by {breakdown} order by cnt1 desc, {breakdown} asc limit 10")
    su = spark.sql(f"select {breakdown}, count(distinct product_key) as cnt2 from db where date='{_date}' group by {breakdown} order by cnt2 desc, {breakdown} asc limit 10")
    sd.join(su, [breakdown], how="left").show()

In [0]:


_date = '2021-05-01'
breakdowns = ['country_code', 'device_code', 'featured_story_key', 'product_key']
for breakdown in breakdowns:
    sd = spark.sql(f"select {breakdown}, sum(featured_item_visibility_score) as sum1 from unified where date='{_date}' group by {breakdown} order by sum1 desc, {breakdown} asc limit 10")
    su = spark.sql(f"select {breakdown}, sum(REACH_SCORE) as sum2 from db where date='{_date}' group by {breakdown} order by sum2 desc, {breakdown} asc limit 10")
    sd.join(su, [breakdown], how="left").show()

In [0]:


_date = '2021-05-01'

for metric in ['reach_score', 'country_code', 'product_key', 'device_code', 'category_item_key', 'collection_label', 'collection_type_code']:
    print(f'{metric}: ', end='')
    spark.sql(f"select count(*) from db where date='{_date}' and {metric} is NULL limit 10").show(1)


In [0]:


sd = spark.sql(f"select * from unified where date='{_date}'")
su = spark.sql(f"select * from db where date='{_date}'")
joined = sd.join(su, ['country_code', 'device_code', 'featured_story_key', 'product_key'], how="left")
assert sd.count() == joined.count()